### Scanner

This notebook contains the code for the scanning system.

Import the required libraries.

In [ ]:
import numpy as np
import pylab as pl
import socket
import time

Connect to the TCP server running on the ALPHA250 board.

In [ ]:
s = socket.create_connection(("192.168.1.100", 1001))

Set the scanner to its home position.

In [ ]:
dac = [0x7FFF, 0x7FFF]

s.send(np.uint64(11 << 32 | dac[1] << 16 | dac[0]))

Move the scanner one small step.

In [ ]:
dac[0] += 100
dac[1] += 100

s.send(np.uint64(11 << 32 | dac[1] << 16 | dac[0]))

Calculate scan coordinates.

In [ ]:
radius = 2000
steps = 50000
turns = 10

l = np.pi * radius * turns
dr = radius / turns
dl = l / steps
x = []
y = []
phi = 0
while len(x) < steps:
    r = phi * dr / (2 * np.pi)
    x.append(int(np.floor(r * np.cos(phi) + 0.5)))
    y.append(int(np.floor(r * np.sin(phi) + 0.5)))
    phi += (np.sqrt(r * r + 2 * dr * dl) - r) / dr

pl.plot(x, y, linestyle="", marker=".", markersize=1)
pl.grid()

Set all configuration parameters.

In [ ]:
scan_step = 10  # us
sample_step = 100  # us

f0 = 40000000  # Hz
f1 = 10000000  # Hz
f2 = 20000000  # Hz
f3 = 30000000  # Hz
f4 = 50000000  # Hz
f5 = 20000000  # Hz
f6 = 60000000  # Hz

s.send(np.uint64(0 << 32 | int(250 * scan_step)))
s.send(np.uint64(1 << 32 | int(sample_step)))
s.send(np.uint64(2 << 32 | f0))
s.send(np.uint64(3 << 32 | f1))
s.send(np.uint64(4 << 32 | f2))
s.send(np.uint64(5 << 32 | f3))
s.send(np.uint64(6 << 32 | f4))
s.send(np.uint64(7 << 32 | f5))
s.send(np.uint64(8 << 32 | f6))

s.send(np.uint64(9 << 32))

for c in zip(x, y):
    s.send(np.uint64(10 << 32 | (c[1] + 0x7FFF) << 16 | (c[0] + 0x7FFF)))

Scan.

In [ ]:
s.send(np.uint64(11 << 32 | 0x7FFF << 16 | 0x7FFF))

time.sleep(0.001)

size = steps * scan_step // sample_step
s.send(np.uint64(12 << 32 | size))


def read_data(data):
    view = data.view(np.uint8)
    offset = 0
    limit = view.size
    while offset < limit:
        buffer = s.recv(limit - offset)
        buffer = np.frombuffer(buffer, np.uint8)
        size = buffer.size
        view[offset : offset + size] = buffer
        offset += size


data0 = np.zeros(7 * size, np.complex64)
data1 = np.zeros(7 * size, np.complex64)

read_data(data0)
read_data(data1)

Plot signal.

In [ ]:
in0 = data0[0::7] # photodiode/ADC0 mixed with LO0
in1 = data0[1::7] # photodiode/ADC0 mixed with LO1
in2 = data0[2::7] # photodiode/ADC0 mixed with LO2
in3 = data0[3::7] # photodiode/ADC0 mixed with LO3
in4 = data0[4::7] # photodiode/ADC0 mixed with LO4
in5 = data0[5::7] # photodiode/ADC0 mixed with LO5
in6 = data0[6::7] # photodiode/ADC0 mixed with LO6

in7 = data1[0::7] # AOM/ADC1 mixed with LO0
in8 = data1[1::7] # EOM1/ADC2 mixed with LO1
in9 = data1[2::7] # EOM2/ADC3 mixed with LO2
in10 = data1[3::7] # AOM/ADC1 mixed with EOM1/ADC2 mixed with LO3
in11 = data1[4::7] # AOM/ADC1 mixed with EOM1/ADC2 mixed with LO4
in12 = data1[5::7] # AOM/ADC1 mixed with EOM2/ADC3 mixed with LO5
in13 = data1[6::7] # AOM/ADC1 mixed with EOM2/ADC3 mixed with LO6

pl.plot(in0.real, in7.real, linestyle="", marker=".", markersize=1)
pl.grid()

Plot spectrum.

In [ ]:
samples = in0

window = np.blackman(samples.size)

fft = np.fft.fft(samples * window) / samples.size * 2
fft = 20 * np.log10(np.abs(fft))
pl.plot(np.fft.fftshift(fft))
pl.grid()

Plot phase.

In [ ]:
phi0 = np.angle(in0, deg=True)
phi0 = np.unwrap(phi0, period=180)

phi7 = np.angle(in7, deg=True)
phi7 = np.unwrap(phi7, period=180)

pl.plot(phi0 - phi7)
pl.ylim(-200, 200)
pl.grid()

Close connection.

In [ ]:
s.close()